# Os dados do Orçamento de Estado de 2021

```{margin} _Caveat emptor!_
A existência de erros e omissões é uma possibilidade a não desprezar: vários foram detectados durante as fases iniciais deste projecto, e alguns nas fases finais. Dada a importância que os dados têm para todas as outras análises o impacto pode ir desde irrelevante até significativo.
```
A base de todos os dados é a informação disponibilizada pela Assembleia da República na secção [Dados Abertos](https://www.parlamento.pt/Cidadania/Paginas/DadosAbertos.aspx). Sobre estes são efectuadas várias transformações que resultam da selecção da informação pretendida; neste processo, e para além das decisões sobre o conteúdo a seleccionar, os dados originais são convertidos em formato tabular.

As análises anteriores mostram segmentos desta informação em forma de tabelas, sempre de tamanho limitado para se enquadrar no fluxo do texto; aqui colocamos um conjunto alargado da informação recolhida sobre as votações do Orçamento de Estado de 2021, essencialmente as Propostas de Alteração e como foram votadas.


In [1]:
!pip3 install --user -q  matplotlib pandas seaborn sklearn 

%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.cluster import SpectralClustering
from sklearn.manifold import MDS
import random
import seaborn as sns
from matplotlib.colors import ListedColormap
sns.set_theme(style="whitegrid", palette="pastel")

In [3]:
from urllib.request import urlopen
import xml.etree.ElementTree as ET

oe_url = "http://app.parlamento.pt/webutils/docs/doc.xml?path=6148523063446f764c324679626d56304c3239775a57356b595852684c3052685a47397a51574a6c636e52766379395063734f6e5957316c626e52764a5449775a47386c4d6a4246633352685a47387657456c574a5449775447566e61584e7359585231636d457654305651636d397762334e3059584e426248526c636d466a595738794d4449785433497565473173&fich=OEPropostasAlteracao2021Or.xml&Inline=true"
oe_tree = ET.parse(urlopen(oe_url))

In [4]:
import collections

counter=0
vc=0
## We will build a dataframe from a list of dicts
## Inspired by the approach of Chris Moffitt here https://pbpython.com/pandas-list-dict.html
oe_list = []

for alter in oe_tree.findall(".//PropostaDeAlteracao"):
    votep = alter.find('./Votacoes')
    if votep is not None:
        oe_dict = collections.OrderedDict()
        counter +=1
        oe_dict["ID"]=alter.find('./ID').text
        oe_dict["Nr"]=alter.find('./Numero').text
        oe_dict["Date"]=alter.find('./Data').text
        oe_dict["Domain"]=alter.find('./Tema').text
        oe_dict["Type"]=alter.find('./Tipo').text
        oe_dict["State"]=alter.find('./Estado').text
        oe_dict["GP"]=alter.find('./GrupoParlamentar_Partido').text
        init_title=alter.find('./Iniciativas_Artigos/Iniciativa_Artigo/Titulo')
        if init_title is not None:
            oe_dict["IniTitle"]=init_title.text
        #oe_list.append(oe_dict)
        for vote in alter.findall("./Votacoes/"):
            vc +=1
            for vote_el in vote:
                if vote_el.tag == "Data":
                    oe_dict["V_Date"] = vote_el.text
                    # print(oe_dict["ID"])
                if vote_el.tag == "Descricoes":
                    descp = vote_el.find('./Descricao')
                    if descp is not None: 
                        oe_dict["VoteDesc"] = vote_el.find('./Descricao').text
                if vote_el.tag == "Resultado":
                    oe_dict["Result"] = vote_el.text
                for gps in vote.findall("./GruposParlamentares/"):
                    if gps.tag == "GrupoParlamentar":
                        gp = gps.text
                    else:
                        oe_dict[gp] = gps.text

        oe_list.append(oe_dict)
    print('.', end='')
        
print("\nProposals:",counter)
print(vc)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
import pandas as pd

oe_df = pd.DataFrame(oe_list)
oe_df

,ID,Nr,Date,Domain,Type,State,GP,IniTitle,VoteDesc,Partido Socialista,...,Pessoas-Animais-Natureza,Chega,Iniciativa Liberal,V_Date,Result,Livre,"Partido Ecologista ""Os Verdes""",CRISTINA RODRIGUES(Ninsc),JOACINE KATAR MOREIRA(Ninsc),SARA MADRUGA DA COSTA(PSD)
0,14658,1440C,13/11/2020 20:08:00,Orçamento e Finanças,Aditamento (Números e Alíneas),Aprovado(a) em Comissão,PS,Investimento em infraestruturas de habitação p...,"N.º 3, Artigo 150.º",Favor,...,Favor,Abstenção,Abstenção,23/11/2020 00:00:00,Aprovado(a),NaN,NaN,NaN,NaN,NaN
1,14656,1439C,13/11/2020 19:59:00,Cultura e Comunicação,Por Definir,Retirado(a),PCP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14655,1438C,13/11/2020 19:59:00,"Educação, Ciência, Juventude e Desporto",Aditamento (Artigo PPL),Aprovado(a) Parcialmente em Comissão,PCP,Programa de apoio ao trabalho artístico e cult...,Artigo 171.º-A,Favor,...,Favor,Favor,Contra,25/11/2020 00:00:00,Aprovado(a),NaN,NaN,NaN,NaN,NaN
3,14654,1437C,13/11/2020 19:59:00,Orçamento e Finanças,Aditamento (Artigo PPL),Rejeitado(a) em Comissão,PCP,Devolução de verbas do Fundo de Garantia Autom...,Artigo 207.º-A,Contra,...,Contra,Abstenção,Contra,24/11/2020 00:00:00,Rejeitado(a),NaN,NaN,NaN,NaN,NaN
4,14653,1436C,13/11/2020 19:58:00,Negócios Estrangeiros e Comunidades Portuguesas,Aditamento (Artigo PPL),Rejeitado(a) em Comissão,PCP,Apoio às associações portuguesas no estrangeiro,Artigo 30.º-A,Contra,...,Favor,Favor,Contra,20/11/2020 00:00:00,Rejeitado(a),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,13152,5C,29/10/2020 09:02:00,Centro de recolha de animais,Emenda,Rejeitado(a) em Comissão,CRISTINA RODRIGUES,"Centros de recolha oficial de animais, apoio à...","N.º 1, Artigo 211.º",Contra,...,Favor,Favor,Favor,24/11/2020 00:00:00,Rejeitado(a),NaN,NaN,NaN,NaN,NaN
1543,13151,4C,29/10/2020 09:02:00,"Ambiente, Energia e Ordenamento do Território",Aditamento (Artigo PPL),Aprovado(a) em Comissão,PEV,Avaliação ambiental estratégica para aferir me...,Artigo 196.º-A,Contra,...,Favor,Favor,Favor,24/11/2020 00:00:00,Aprovado(a),NaN,NaN,NaN,NaN,NaN
1544,13150,3C,29/10/2020 09:01:00,Creches,Aditamento (Números e Alíneas),Rejeitado(a) em Comissão,PCP,Gratuitidade de creche,"N.º 7, Artigo 114.º",Contra,...,Favor,Abstenção,Contra,20/11/2020 00:00:00,Rejeitado(a),NaN,NaN,NaN,NaN,NaN
1545,13149,2C,29/10/2020 09:01:00,"Administração Pública, Modernização Administra...",Substituição,Aprovado(a) em Comissão,PCP,Reforço de dotação do pessoal não docente na e...,Artigo 174.º,Favor,...,Favor,Favor,Contra,23/11/2020 00:00:00,Aprovado(a),NaN,NaN,NaN,NaN,NaN


In [6]:
import collections

counter=0
vc=0
## We will build a dataframe from a list of dicts
## Inspired by the approach of Chris Moffitt here https://pbpython.com/pandas-list-dict.html
oe_list = []

for alter in oe_tree.findall(".//PropostaDeAlteracao"):
    votep = alter.find('./Votacoes')
    if votep is not None:
        oe_dict = collections.OrderedDict()
        counter +=1
        oe_dict["ID"]=alter.find('./ID').text
        oe_dict["Nr"]=alter.find('./Numero').text
        oe_dict["Date"]=alter.find('./Data').text
        oe_dict["Domain"]=alter.find('./Tema').text
        oe_dict["Type"]=alter.find('./Tipo').text
        oe_dict["State"]=alter.find('./Estado').text
        oe_dict["GP"]=alter.find('./GrupoParlamentar_Partido').text
        init_title=alter.find('./Iniciativas_Artigos/Iniciativa_Artigo/Titulo')
        if init_title is not None:
            oe_dict["IniTitle"]=init_title.text
        #oe_list.append(oe_dict)
        for vote in alter.findall("./Votacoes/"):
            vc +=1
            for vote_el in vote:
                if vote_el.tag == "Data":
                    oe_dict["V_Date"] = vote_el.text
                    # print(oe_dict["ID"])
                if vote_el.tag == "Descricoes":
                    descp = vote_el.find('./Descricao')
                    if descp is not None: 
                        oe_dict["VoteDesc"] = vote_el.find('./Descricao').text
                if vote_el.tag == "SubDescricao":
                    oe_dict["SubDesc"] = vote_el.text
                if vote_el.tag == "Resultado":
                    oe_dict["Result"] = vote_el.text
                for gps in vote.findall("./GruposParlamentares/"):
                    if gps.tag == "GrupoParlamentar":
                        gp = gps.text
                    else:
                        oe_dict[gp] = gps.text

            oe_list.append(oe_dict)
    print('.', end='')
        
print("\nProposals:",counter)
print("Voting sessions:", vc)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Proposals: 1547
Voting sessions: 1646


In [7]:
import pandas as pd

oe_df = pd.DataFrame(oe_list)
oe_df
oe_df["VoteDesc"] = oe_df["VoteDesc"] + ": " + oe_df["SubDesc"]
oe_df.drop(['SubDesc'],axis=1,inplace=True)
oe_df = oe_df.fillna("")
oe_df

,ID,Nr,Date,Domain,Type,State,GP,IniTitle,VoteDesc,Partido Socialista,...,Pessoas-Animais-Natureza,Chega,Iniciativa Liberal,V_Date,Result,Livre,"Partido Ecologista ""Os Verdes""",CRISTINA RODRIGUES(Ninsc),JOACINE KATAR MOREIRA(Ninsc),SARA MADRUGA DA COSTA(PSD)
0,14658,1440C,13/11/2020 20:08:00,Orçamento e Finanças,Aditamento (Números e Alíneas),Aprovado(a) em Comissão,PS,Investimento em infraestruturas de habitação p...,,Favor,...,Favor,Abstenção,Abstenção,23/11/2020 00:00:00,Aprovado(a),,,,,
1,14655,1438C,13/11/2020 19:59:00,"Educação, Ciência, Juventude e Desporto",Aditamento (Artigo PPL),Aprovado(a) Parcialmente em Comissão,PCP,Programa de apoio ao trabalho artístico e cult...,Artigo 171.º-A: Programa de apoio ao trabalho ...,Favor,...,Favor,Favor,Contra,25/11/2020 00:00:00,Aprovado(a),,,,,
2,14654,1437C,13/11/2020 19:59:00,Orçamento e Finanças,Aditamento (Artigo PPL),Rejeitado(a) em Comissão,PCP,Devolução de verbas do Fundo de Garantia Autom...,Artigo 207.º-A: Devolução de verbas do Fundo d...,Contra,...,Contra,Abstenção,Contra,24/11/2020 00:00:00,Rejeitado(a),,,,,
3,14653,1436C,13/11/2020 19:58:00,Negócios Estrangeiros e Comunidades Portuguesas,Aditamento (Artigo PPL),Rejeitado(a) em Comissão,PCP,Apoio às associações portuguesas no estrangeiro,Artigo 30.º-A: Apoio às associações portuguesa...,Contra,...,Favor,Favor,Contra,20/11/2020 00:00:00,Rejeitado(a),,,,,
4,14652,1435C,13/11/2020 19:58:00,Assuntos Europeus,Aditamento (Artigo PPL),Rejeitado(a) em Comissão,PCP,Apoio à retoma das atividades das associações ...,Artigo 164.º-A: Apoio à retoma das atividades ...,Contra,...,Favor,Favor,Abstenção,23/11/2020 00:00:00,Rejeitado(a),,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,13150,3C,29/10/2020 09:01:00,Creches,Aditamento (Números e Alíneas),Rejeitado(a) em Comissão,PCP,Gratuitidade de creche,,Contra,...,Favor,Abstenção,Contra,20/11/2020 00:00:00,Rejeitado(a),,,,,
1642,13150,3C,29/10/2020 09:01:00,Creches,Aditamento (Números e Alíneas),Rejeitado(a) em Comissão,PCP,Gratuitidade de creche,,Contra,...,Favor,Abstenção,Contra,20/11/2020 00:00:00,Rejeitado(a),,,,,
1643,13150,3C,29/10/2020 09:01:00,Creches,Aditamento (Números e Alíneas),Rejeitado(a) em Comissão,PCP,Gratuitidade de creche,,Contra,...,Favor,Abstenção,Contra,20/11/2020 00:00:00,Rejeitado(a),,,,,
1644,13149,2C,29/10/2020 09:01:00,"Administração Pública, Modernização Administra...",Substituição,Aprovado(a) em Comissão,PCP,Reforço de dotação do pessoal não docente na e...,Artigo 174.º: Reforço de dotação do pessoal nã...,Favor,...,Favor,Favor,Contra,23/11/2020 00:00:00,Aprovado(a),,,,,


In [8]:
oe_df.to_json("oe2021.json", orient="records")

In [9]:
from IPython.display import display, HTML
js = '''
<div>
    <button id="download-csv">Download CSV</button>
    <button id="download-json">Download JSON</button>
</div>

<div id="example-table"></div>
<script>
var table = new Tabulator("#example-table", {
    pagination:"local",
    responsiveLayout:true,
    paginationSize:50,         //allow 20 rows per page of data
    movableColumns:true,      //allow column order to be changed
    resizableRows:true,       //allow row order to be changed
    autoColumns:true,
    layout:"fitColumns",
   // ajaxURL:"http://0.0.0.0:8000/all_votes.json", //ajax URL
    ajaxURL:"./oe2021.json", //ajax URL
});

//trigger download of data.csv file
document.getElementById("download-csv").addEventListener("click", function(){
    table.download("csv", "data.csv");
});

//trigger download of data.json file
document.getElementById("download-json").addEventListener("click", function(){
    table.download("json", "data.json");
});

</script>
'''
display(HTML(js))